# Importing data from Yahoo Finance for S&P500 and MSCI World, and calculating data moments

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
from scipy.stats import skew, kurtosis
import warnings

warnings.filterwarnings("ignore")

# SET risk-free rate here
rf = 0.039
alpha = 1.0  # full investment

# Fetch data
tickers = ['SPY', 'URTH']
data = yf.download(tickers, start='1993-01-01', end='2024-12-31')['Close'].resample('M').last()

# Compute log returns per ticker independently
log_returns = pd.concat([
    np.log(data[ticker] / data[ticker].shift(1)).rename(ticker)
    for ticker in tickers
], axis=1)

n = log_returns.count()

# Annualized moments
mean_log = log_returns.mean() * 12
variance_log = log_returns.var() * 12
volatility_log = np.sqrt(variance_log)
skewness = log_returns.apply(skew, nan_policy='omit')
kurtosis_ = log_returns.apply(kurtosis, nan_policy='omit', fisher=False)

# Standard errors
se_mean = volatility_log / np.sqrt(n)
se_variance = variance_log * np.sqrt(2 / n)
se_volatility = se_variance / (2 * volatility_log)
se_skewness = np.sqrt(6 / n)
se_kurtosis = np.sqrt(24 / n)

# Tables
moments_df = pd.DataFrame({
    'Mean': mean_log,
    'Variance': variance_log,
    'Volatility': volatility_log,
    'Skewness': skewness,
    'Kurtosis': kurtosis_
}, index=tickers)

se_df = pd.DataFrame({
    'Mean SE': se_mean,
    'Variance SE': se_variance,
    'Volatility SE': se_volatility,
    'Skewness SE': se_skewness,
    'Kurtosis SE': se_kurtosis
}, index=tickers)

ci_df = pd.DataFrame({
    'Mean 95% CI': [f"({mean_log[i] - 1.96 * se_mean[i]:.4f}, {mean_log[i] + 1.96 * se_mean[i]:.4f})" for i in range(len(tickers))],
    'Variance 95% CI': [f"({variance_log[i] - 1.96 * se_variance[i]:.4f}, {variance_log[i] + 1.96 * se_variance[i]:.4f})" for i in range(len(tickers))],
    'Volatility 95% CI': [f"({volatility_log[i] - 1.96 * se_volatility[i]:.4f}, {volatility_log[i] + 1.96 * se_volatility[i]:.4f})" for i in range(len(tickers))],
    'Skewness 95% CI': [f"({skewness[i] - 1.96 * se_skewness[i]:.4f}, {skewness[i] + 1.96 * se_skewness[i]:.4f})" for i in range(len(tickers))],
    'Kurtosis 95% CI': [f"({kurtosis_[i] - 1.96 * se_kurtosis[i]:.4f}, {kurtosis_[i] + 1.96 * se_kurtosis[i]:.4f})" for i in range(len(tickers))]
}, index=tickers)

# Construct gamma input
gamma_input = pd.DataFrame({
    'ticker': tickers,
    'rf': rf,
    'alpha': alpha,
    'er_excess': mean_log - rf,
    'sigma': volatility_log,
    'skew': skewness,
    'kurt': kurtosis_,
    'er_excess_lower': mean_log - rf - 1.96 * se_mean,
    'er_excess_upper': mean_log - rf + 1.96 * se_mean,
    'sigma_lower': volatility_log - 1.96 * se_volatility,
    'sigma_upper': volatility_log + 1.96 * se_volatility
}).reset_index(drop=True)

# Print tables
print("=== Moments ===")
print(moments_df.round(4))
print("\n=== Standard Errors ===")
print(se_df.round(4))
print("\n=== 95% Confidence Intervals ===")
print(ci_df)

[*********************100%***********************]  2 of 2 completed

=== Moments ===
        Mean  Variance  Volatility  Skewness  Kurtosis
SPY   0.0996    0.0223      0.1493   -0.7516    4.3397
URTH  0.1052    0.0204      0.1428   -0.5961    3.9638

=== Standard Errors ===
      Mean SE  Variance SE  Volatility SE  Skewness SE  Kurtosis SE
SPY    0.0076       0.0016         0.0054       0.1252       0.2503
URTH   0.0115       0.0023         0.0081       0.1967       0.3935

=== 95% Confidence Intervals ===
           Mean 95% CI   Variance 95% CI Volatility 95% CI  \
SPY   (0.0846, 0.1145)  (0.0191, 0.0254)  (0.1387, 0.1599)   
URTH  (0.0827, 0.1276)  (0.0158, 0.0249)  (0.1269, 0.1586)   

         Skewness 95% CI   Kurtosis 95% CI  
SPY   (-0.9969, -0.5063)  (3.8490, 4.8303)  
URTH  (-0.9817, -0.2104)  (3.1925, 4.7350)  


# Investigating the data

In [136]:
first_valid = data.apply(lambda col: col.first_valid_index())
print(first_valid)

valid_counts = data.count()
print(valid_counts)

Ticker
SPY    1993-01-31
URTH   2012-01-31
dtype: datetime64[ns]
Ticker
SPY     384
URTH    156
dtype: int64


# Estimating risk aversion

In [142]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize_scalar

# --- Utility function ---
def expected_utility(alpha, gamma, rf, er_excess, sigma, skew, kurt):
    base = 1 + rf + alpha * er_excess
    if base <= 0:
        return -np.inf  # Invalid utility

    term1 = (base ** (1 - gamma)) / (1 - gamma)
    correction2 = -(gamma / 2) * (sigma ** 2) * (alpha ** 2) * base ** (-gamma - 1)
    correction3 = (gamma * (1 + gamma) / 6) * (sigma ** 3) * (alpha ** 3) * skew * base ** (-gamma - 2)
    correction4 = -(gamma * (1 + gamma) * (2 + gamma) / 24) * (sigma ** 4) * (alpha ** 4) * kurt * base ** (-gamma - 3)

    return term1 + correction2 + correction3 + correction4

# --- Objective function ---
def crra_objective(gamma, rf, er_excess, sigma, skew, kurt, alpha):
    return -expected_utility(alpha, gamma, rf, er_excess, sigma, skew, kurt)

# --- Gamma estimator ---
def estimate_crra(rf, sigma, er_excess, skew, kurt, alpha=1.0):
    result = minimize_scalar(
        crra_objective,
        method='brent',
        bracket=(2, 7),
        args=(rf, er_excess, sigma, skew, kurt, alpha)
    )
    return result.x if result.success else np.nan

# --- Run estimation for both tickers ---
results = []
for _, row in gamma_input.iterrows():
    rf = float(row['rf'])
    alpha = float(row['alpha'])

    gamma_mid = estimate_crra(rf, float(row['sigma']), float(row['er_excess']), float(row['skew']), float(row['kurt']), alpha)
    gamma_low = estimate_crra(rf, float(row['sigma_upper']), float(row['er_excess_lower']), float(row['skew']), float(row['kurt']), alpha)
    gamma_high = estimate_crra(rf, float(row['sigma_lower']), float(row['er_excess_upper']), float(row['skew']), float(row['kurt']), alpha)

    results.append({
        'Ticker': row['ticker'],
        'Gamma (Lower CI)': gamma_low,
        'Gamma (Estimate)': gamma_mid,
        'Gamma (Upper CI)': gamma_high
    })

# Output
gamma_estimates = pd.DataFrame(results)
print("\n=== CRRA Gamma Estimates ===")
print(gamma_estimates.round(4))



=== CRRA Gamma Estimates ===
  Ticker  Gamma (Lower CI)  Gamma (Estimate)  Gamma (Upper CI)
0    SPY            8.9352           11.0344        14301.4086
1   URTH            9.2894           13.8021        12666.5951


In [ ]:
# ---------------- Sharpe ratios ----------------
# Point estimate
gamma_input["sharpe_est"] = gamma_input["er_excess"] / gamma_input["sigma"]

# Worst-case within the 95 % CI
gamma_input["sharpe_low"] = gamma_input["er_excess_lower"] / gamma_input["sigma_upper"]

# Best-case within the 95 % CI
gamma_input["sharpe_high"] = gamma_input["er_excess_upper"] / gamma_input["sigma_lower"]

# Collect the three ratios in a table
sharpe_cols = ["ticker", "sharpe_low", "sharpe_est", "sharpe_high"]
sharpe_estimates = gamma_input[sharpe_cols].rename(columns={
    "ticker": "Ticker",
    "sharpe_low": "Sharpe (Lower CI)",
    "sharpe_est": "Sharpe (Estimate)",
    "sharpe_high": "Sharpe (Upper CI)"
})

print("\n=== Annualised Sharpe Ratios ===")
print(sharpe_estimates.round(4))



=== Annualised Sharpe Ratios ===
  Ticker  Sharpe (Lower CI)  Sharpe (Estimate)  Sharpe (Upper CI)
0    SPY             0.2852             0.4056             0.5443
1   URTH             0.2754             0.4635             0.6987
